In [10]:
import pandas as pd 
import glob
import xarray as xr
import time

In [70]:
path = "data/101"

more_files = sorted(glob.glob(path + "/*.dat"))[:10] 
#prøverunnde med noen filer. denne finner alle filene som slutterpå ".dat" og leser bare de 10 første
#ikke utvit til noe særlig mer en typ 20-30 stasjoner. da sprenger pcen. burde gå bedre når vi effektiviserer etterpå


first = True
start = time.time()
for i in more_files:
    
    df = pd.read_table(i,skiprows=2, delim_whitespace=True,skipfooter=1,engine="python")
    
    
    if(df.values.shape[0]<10): # 
        continue
    
    meta = pd.read_table(i,skiprows=None,sep="\s+",nrows=1,engine="python")
    print(df.values.shape[0], int(str(meta.head().columns[3])[:-1]))
    
    measurement_time = pd.to_datetime(float(meta.values[0,1]),origin=str(int(meta.values[0,0])),unit="D")
    measurement_lat  = float(meta.values[0,2])
    measurement_lon  = float(meta.values[0,3])
    #print(measurement_time)
    
    
    if "%year" in df.columns:
        df = df.drop(["%year","day"],axis=1)
    if "nobs" in df.columns:
        df = df.drop("nobs",axis=1)
    #if "dissolved_oxygen" in df.columns:
        #df = df.rename(columns={"dissolved_oxygen"})
    
    df.rename(columns={"%pressure(dbar)":"pressure","pressure(dbar)":"pressure","temperature(C)":"temperature"}, inplace=True)
    df = df.set_index("pressure")

    ds = xr.Dataset.from_dataframe(df)
    

    ds["time"] = measurement_time
    ds["lat"]  = measurement_lat
    ds["lon"]  = measurement_lon
    
    #profile_nr = int(list(pd.read_table(i,sep="[:, ]",nrows=0,engine="python"))[4])
    profile_nr = int(str(meta.head().columns[3])[:-1])
    
    
    ds = ds.assign_coords(profile=profile_nr)
    ds = ds.expand_dims("profile")

    
    # joining files
    if first==True:
        itp114 = ds
        first=False
    else:
        itp114=xr.concat([itp114,ds],dim = "profile")
        #concat gjør at det blir mye nan i temp og salinity, men det følger cf. hør med Steingod

print("Det tok", time.time()-start)

#La oss fikse litt metadata da
#legg til alle de her greiene: https://adc.met.no/node/4

itp114.time.attrs["standard_name"] = "time"
itp114.time.attrs["units"] = "seconds since 1970-01-01 00:00:00+0"# husk å faktisk endre til dette formatet
itp114["temperature"].attrs["standard_name"] = "sea_water_temperature"
itp114["temperature"].attrs["units"] = "celcius" # husk å bytte til kelvin
itp114["pressure"].attrs["standard_name"] = "sea_water_pressure"
itp114["pressure"].attrs["units"] = "dbar"
itp114["dissolved_oxygen"].attrs["standard_name"] = "moles_of_oxygen_per_unit_mass_in_sea_water"
itp114["dissolved_oxygen"].attrs["units"] = "umol/kg"

itp114.attrs["featureType"] = "trajectoryProfile"

itp114.attrs

776 1
753 2
751 3
753 4
754 5
754 6
750 7
753 8
762 9
754 10
Det tok 0.2180626392364502


{'featureType': 'trajectoryProfile'}

Info:
====
**infoen er forløpig basert på gjennomgang av bare en bøye + at jeg har sett litt på hva som står om de andre. Så den kan være ugenerell, oppdaterer fortløpende. **


Data fra bøyene inholder profiler med trykk, temp og salt. 
Hver profil har en .dat fil med en header som inneholder starttid for profilen, lat,long og hvor mange målinger fila inneholder. enhet for tid er ÅR DAG, der dag har desimaler som gir klokkeslett. dag begynner på 1, ikke null. 

Noen bøyer har også andre målinger: bøya jeg nå ser på, 114, har to fastmonterte microcatsensorer som måler på 5 og 6 meters dyp hvert kvarter. de ligger i filene som har micro i navnet og er mye større. 

Jobber i første omgang med å lage et program som henter data fra én profil og en bøye.


Lage et program som dumper dette til netCDF-CF. 


Tanker og Spørsmål:
============



Ser ut til at metadata og discover-metadata må hardcodes for å stemme med standarene for [metadata]() og [discover-metadata](https://adc.met.no/node/4)

Skal jeg bruke trajectoryProfile eller Profile

skal jeg inkludere mlinger fra f.eks microcat-sensorene 